# Wav2Vec2를 사용한 음성 인식

**Colab 사용**

wav2vec 2.0의 사전 학습된 모델을 사용하여 음성 인식을 수행하는 방법

## 개요

음성 인식 프로세스는 다음과 같습니다.

1. 오디오 파형에서 음향 특징 추출 .
2. 딥러닝 모델을 이용하여 음향 특징의 프레임별로 음성 클래스 분류  
3. 클래스 확률 시퀀스에서 가설 생성 --> 예측된 소리를 연결해서 실제 단어로 만드는 과정 (CTC 디코딩 기법)




VOiCES (Voices Obscured in Complex Environmental Settings)  
- 환경 소음이 있는 다양한 상황에서의 음성을 포함한 데이터셋

In [ ]:
# VOiCES 음성 파일 다운로드

## 파이프라인 만들기

먼저, 특징 추출과 분류를 수행하는 Wav2Vec2 모델을 만듭니다.

torchaudio에서 사용할 수 있는 Wav2Vec2 사전 학습된 가중치에는 두 가지 유형이 있습니다. ASR(Automatic Speech Recognition) 작업에 맞게 미세 조정된 가중치와 미세 조정되지 않은 가중치입니다.

Wav2Vec2(및 HuBERT) 모델은 자체 감독 방식으로 학습됩니다. 먼저 표현 학습을 위해 오디오로만 학습한 다음 추가 레이블이 있는 특정 작업에 맞게 미세 조정합니다.

번들 객체는 모델 및 기타 정보를 인스턴스화하는 인터페이스를 제공합니다.



In [ ]:
# 사전 학습된 Wav2Vec2 모델을 가져옵니다. 이 모델은 960시간의 영어 음성 데이터를 학습한 모델입니다.
# 샘플 레이트를 출력합니다. 오디오 데이터를 이 샘플 레이트로 변환해야 모델이 올바르게 처리할 수 있습니다.
# 모델에서 사용하는 레이블을 출력합니다. 이 레이블은 각 음소나 단어에 대응하는 클래스입니다.
# 모델이 음성 신호에서 예측한 각 프레임에 대해 어떤 클래스나 문자를 사용할지 정의하는 라벨의 목록을 반환

모델은 다음과 같이 구성할 수 있습니다. 이 프로세스는 사전 훈련된 가중치를 자동으로 가져와 모델에 로드합니다.




In [ ]:
# 모델을 가져오고, 'device'로 전송합니다. .
# 모델의 클래스 정보를 출력합니다. 이 모델이 어떤 종류의 신경망인지 알 수 있습니다.

## 데이터 로딩

Creative Commos BY 4.0 라이선스를 받은 [VOiCES 데이터 세트](https://iqtlabs.github.io/voices/)_의 음성 데이터를 사용합니다.




데이터를 로드하려면 `torchaudio.load` 함수를 사용합니다.

샘플링 속도가 파이프라인에서 기대하는 것과 다르면, `torchaudio.functional.resample` 함수를 사용하여 리샘플링을 할 수 있습니다.




In [ ]:
# 음성 파일(SPEECH_FILE)을 로드하여 waveform과 sample_rate(샘플링 속도)를 가져옴

In [ ]:
# 로드한 waveform을 지정한 device로 이동
# 현재 샘플링 속도가 모델(bundle)이 요구하는 샘플링 속도와 다를 경우, 리샘플링 수행
    # torchaudio.functional.resample 함수를 사용하여 waveform의 샘플링 속도를 변환

## 음성 특징 추출

오디오에서 음향 특징을 추출



In [ ]:
# 추론 모드에서 실행 (모델 가중치 업데이트 방지)
    # 모델을 사용하여 waveform에서 음성 특징(features) 추출
    # 두 번째 반환값(_)은 필요 없으므로 무시

반환된 특징(features)은 텐서들의 리스트입니다. 각 텐서는 트랜스포머 레이어의 출력입니다.

- 첫 번째 차원: 배치 크기 (batch size).
1은 이 텐서가 한 번에 하나의 음성 데이터(또는 입력)를 처리했음을 의미  
- 두 번째 차원: 시퀀스 길이 또는 프레임 수
Transformer 모델에서 시퀀스의 각 시간 단계(프레임)에 대한 특징을 추출한 것. 예를 들어 169는 음성 신호가 169개의 프레임으로 나뉘어 각각에 대해 특징이 계산된 것을 의미.  
- 세 번째 차원: 특징 차원 (feature dimension)  추출된 특징 벡터의 크기.  임베딩 차원.



## 특징 분류

음향 특징이 추출되면, 다음 단계는 이를 여러 카테고리로 분류하는 것입니다.

Wav2Vec2 모델은 특징 추출과 분류를 한 단계에서 수행하는 메서드를 제공합니다.




In [ ]:
# 추론 모드에서 실행 (모델 가중치 업데이트 방지)
    # 모델을 사용하여 waveform에서 분류 결과(emission)와 추가적인 출력값(_)을 생성
    # 두 번째 반환값(_)은 필요 없으므로 무시

출력은 로짓(logit) 형태로 제공됩니다.

1: 배치 크기 (한 번에 처리된 입력 음성 데이터의 개수)  
169: 음성 신호가 나누어진 프레임의 개수 (시간적 시퀀스 길이)  
29: 각 프레임에 대해 예측할 수 있는 클래스(음소 또는 문자)의 개수  

모델이 169개의 프레임에 대해 각각 29개의 클래스 중 하나를 예측하는 logit 또는 분류 결과를 생성했다는 것을 의미  




In [ ]:
# 분류 결과를 이미지로 시각화 (emission의 첫 번째 배치를 사용)
# emission[0]을 CPU로 이동한 후 전치(transpose)하여 시각화
# x축과 y축에 레이블 설정 (x축은 시간 축, y축은 클래스)

우리는 시간 축을 따라 특정 라벨에 대한 강한 신호가 나타나는 것을 볼 수 있습니다.




## 전사(Transcription) 생성

전사(transcript)는 음성 데이터를 텍스트로 변환한 결과를 의미합니다.

라벨 확률의 시퀀스로부터 이제 전사를 생성하려고 합니다. 가설을 생성하는 과정은 종종 "디코딩"이라고 불립니다.

디코딩은 단순한 분류보다 더 정교한 작업입니다. 왜냐하면 특정 시간 단계에서의 디코딩이 주변 관찰에 영향을 받을 수 있기 때문입니다.

예를 들어, "night"와 "knight"라는 단어를 생각해보면, 일반적인 대화에서 "night"가 "knight"보다 훨씬 자주 나타나지만, "knight with a sword"와 같은 문맥에서 정확하게 "knight"를 전사하려면 디코딩 과정에서 충분한 문맥을 볼 때까지 최종 결정을 미뤄야 합니다.

디코딩 기술은 여러 가지가 있으며, 이들은 외부 자원(예: 단어 사전과 언어 모델)을 필요로 합니다.

여기서는 단순화를 위해 외부 구성 요소에 의존하지 않고, 각 시간 단계에서 가장 좋은 가설을 선택하는 그리디(greedy) 디코딩을 수행할 것입니다. 따라서 문맥 정보는 사용되지 않으며, 하나의 전사만 생성될 수 있습니다.

먼저 그리디 디코딩 알고리즘을 정의하는 것부터 시작하겠습니다.


In [ ]:
# 모델에 설정된 클래스 라벨을 출력

In [ ]:
# emission에서 가장 높은 값을 가진 라벨 인덱스

In [ ]:
# 연속된 동일한 인덱스를 제거하여 유일한 라벨 시퀀스만 남김

In [ ]:
# 공백 토큰을 제외한 인덱스만 남김

In [ ]:
# 라벨 인덱스를 라벨 문자열로 변환하여 결합

위 스크립트들을 묶어 하나의 전사 생성 함수로 만듭니다.

In [ ]:
class GreedyCTCDecoder(torch.nn.Module):
    def __init__(self, labels, blank=0):
        # 그리디 CTC 디코더 클래스 초기화
        # labels: 라벨 목록
        # blank: 공백 토큰의 인덱스 (CTC에서 사용하는 공백 라벨)
    def forward(self, emission: torch.Tensor) -> str:
        # emission에서 가장 높은 값을 가진 라벨 인덱스를 찾음
        # 연속된 동일한 인덱스를 제거하여 유일한 라벨 시퀀스만 남김
        # 공백 토큰을 제외한 인덱스만 남김
        # 라벨 인덱스를 라벨 문자열로 변환하여 결합한 후 반환

이제 디코더 객체를 생성하고 label을 디코딩합니다.

In [ ]:
# 모델에 설정된 라벨을 사용하여 그리디 CTC 디코더를 초기화
# emission의 첫 번째 배치를 디코더에 전달하여 전사(Transcript)를 생성

결과를 확인하고 오디오를 다시 들어보겠습니다.


